# Суммаризация информации из таблиц по DocX документам

### Скачай все необходимые модели для работы

- Sentence:	[sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2)

- NLP:		[ru_core_news_sm](https://spacy.io/models/ru)

- LLM:		[t-tech/T-lite-it-1.0](https://huggingface.co/t-tech/T-lite-it-1.0)

In [ ]:
# LLM

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "t-tech/T-lite-it-1.0"

bnb_config = BitsAndBytesConfig(
	load_in_4bit=True,
	bnb_4bit_quant_type="nf4",
	bnb_4bit_compute_dtype=torch.float16, # Используем torch.float16 вместо 'fp16'
	bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
	model_name,
	quantization_config=bnb_config
)

model.save_pretrained("_models/llm/")
tokenizer.save_pretrained("_models/llm/")

In [ ]:
# NLP

import spacy

nlp = spacy.load('ru_core_news_sm')
nlp.to_disk("_models/nlp/")

In [ ]:
# Sentence

import torch
from transformers import AutoTokenizer, AutoModel

EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
model = AutoModel.from_pretrained(EMBEDDING_MODEL_NAME)

model.save_pretrained("_models/sentence/")
tokenizer.save_pretrained("_models/sentence/")

### Загрузи твою Embedding модель в память. Она будет искать подходящие таблицы под твой запрос

In [1]:
# Загрузка модели поиска данных
from lib.embedding import EmbeddingModel

embedding_model = EmbeddingModel(model_name="_models/sentence/",half=True)

### Загрузи готовую базу данных с документами из Русской википедии

Предварительно скачай её по [ссылке](https://drive.google.com/drive/folders/1-Tf7lbhTVmndmB5xKGTTE5r3eWNF8mih?usp=drive_link)

Здесь выведем, сколько таблиц хранится в БД

In [2]:
# Загрузка БД википедии
from lib.embedding import create_or_load_faiss_index

vector_store = create_or_load_faiss_index("_database/ru_wikipedia",embedding_model)

vector_store.index.ntotal

342658

### Или создай свой индекс и загрузи туда свои документы

Не забудь про NLP модель для обработки контекста твоих документов 🙂

In [4]:
# Загрузка NLP модели
import spacy

nlp = spacy.blank("ru")
nlp.from_disk("_nlp/")
nlp.add_pipe('sentencizer')

In [ ]:
from lib.embedding import create_empty_faiss_index
from lib.nlp import docx2faiss

vector_store = create_empty_faiss_index(embedding_model)

docx2faiss(["_documents/Бирюлево_Быкова_ред.docx","_documents/отчет по выполнению муниципального задания во 2 квартале 2020 г.квартал.docx"],vector_store,nlp)

vector_store.save_local("_database/data/")

### Найди k наиболее похожих результатов по косинусному расстоянию

Здесь применяется алгоритм KNN 😎

Выведем эту таблицу с её контекстом

In [3]:
# Теперь вы можете сделать запрос на похожие документы
query = "Заключенные в гулагах"
docs = vector_store.similarity_search(query, k=5)  # k - сколько результатов получить

for doc in docs:
	print(doc.metadata ,":\n", doc.page_content)

{'uuid': 'wiki-1035831-Федеральная_служба_исполнения_наказаний-315858', 'header': ['|Год', '|Общая численность заключённых', '|Доля заключённых на 100 тыс. населения']} :
 Федеральная служба исполнения наказаний
* «Росомаха» — отдел специального назначения УФСИН России по Ямало-Ненецкому автономному округу;
Статистика
Тенденция снижения численности заключенных, 2000—2023
|Год | |Общая численность заключённых | |Доля заключённых на 100 тыс. населения
|Год | |Общая численность заключённых | |Доля заключённых на 100 тыс. населения
2000 | 1 060 404 | 729
2002 | 980 151 | 675
2004 | 847 004 | 588
2006 | 823 403 | 577
2008 | 883 436 | 622
2010 | 864 197 | 609
2012 | 755 651 | 528
2014 | 677 287 | 471
2016 | 646 085 | 448
2018 | 602 176 | 416
2020 | 523 928 | 363
2021 | 472 226 | 327
2022 | 468 237 | 324
2023 | 433 000 | 300
{'uuid': 'wiki-2019735-Азербайджан_в_Великой_Отечественной_войне-608875', 'header': ['Лагерь военнопленных', 'Госпиталь', 'Медсанчасть']} :
 Азербайджан в Великой О

И ещё один запрос

In [4]:
query = "Население Литвы"
docs = vector_store.similarity_search(query, k=5)

for doc in docs:
	print(doc.metadata ,":\n", doc.page_content)

{'uuid': 'wiki-6623782-Литовцы_в_Литве-2003725', 'header': ['', 'Численность', '%']} :
 Литовцы в Литве
Численность и доля
Переписи населения
Численность и доля литовцев по данным переписи населения за эти годы:

 | Численность | %
 | Численность | %
1959 | 2 150 767 | 79,33
1970 | 2 506 751 | 80,13
1979 | 2 712 233 | 79,97
1989 | 2 924 251 | 79,57
2001 | 2 907 293 | 83,44
2011 | 2 561 314 | 84,15
{'uuid': 'wiki-6623448-Латыши_в_Литве-2003596', 'header': ['', 'Численность', '%']} :
 Латыши в Литве
Численность и доля
Переписи населения
Численность и доля латышей по данным переписи населения за эти годы:

 | Численность | %
 | Численность | %
1959 | 6 318 | 0.23
1970 | 5 063 | 0.16
1979 | 4 354 | 0.12
1989 | 4 229 | 0.11
2001 | 2 955 | 0.08
2011 | 2 025 | 0.06
{'uuid': 'wiki-2883424-Мадонский_край-872170', 'header': ['Национальность', 'Численность,чел. (2011)', '%']} :
 Мадонский край
По оценке на 1 января 2015 года население края составляло 23 778 постоянных жителей, на 1 января 2010 

Ну и на последок

In [8]:
query = "Государственный долг США"
docs = vector_store.similarity_search(query, k=5)

for doc in docs:
	print(doc.metadata ,":\n", doc.page_content)

{'uuid': 'wiki-163538-Государственный_долг_США-72796', 'header': ['Год', 'Госдолг США, млрд $', 'ВВП, млрд $', 'Доля госдолга от ВВП, %']} :
 Государственный долг США
15 ноября 2024 года сумма государственного долга США впервые в истории превысила $36 трлн.
История
Историческая динамика долга
Госдолг США по годам на 31 декабря
Год | Госдолг США, млрд $ | ВВП, млрд $ | Доля госдолга от ВВП, %
Год | Госдолг США, млрд $ | ВВП, млрд $ | Доля госдолга от ВВП, %
1910 | 2,7 | 33,2 | 8,1
1920 | 26,0 | 89,0 | 29,2
1930 | 16,2 | 98,4 | 16,5
1940 | 50,7 | 98,2 | 51,6
1950 | 256,9 | 279,0 | 94,0
1960 | 290,5 | 535,1 | 54,3
1970 | 380,9 | 1049,1 | 36,3
1980 | 909,0 | 2796,8 | 32,5
1990 | 3206,3 | 5914,6 | 54,2
2000 | 5628,7 | 10148,2 | 55,5
2010 | 13 528,7 | 14 798,5 | 91,4
2017 | 20 164,0 | 19 362,1 | 104,1
2020 | 26 000,0 | 20 600,0 | 126,2
2022 | 31 000,0 | 23 000,0 | 134,8
2024 | 35 000,0 | 26 940,0 | 129,8
{'uuid': 'wiki-391154-Экономика_США-154919', 'header': ['Страна', 'Млрд. долларов', '%']

### Загрузи языковую модель для суммаризации ответа по найденному контексту

In [5]:
from lib.llm import LLM

llm = LLM("_models/llm/")

Unused kwargs: ['low_cpu_mem_usage']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
llm.answer(docs[0].page_content,query)

'По состоянию на 15 ноября 2024 года государственный долг США составил более $36 трлн. (Источник: Контекст)'